## Introduction to quantization - PyTorch Python API

Per quantizzazione si intende quella tecnica che consente di eseguire computazioni e memorizzare tensori usando un numero di bit inferiore rispetto al caso in cui i tensori siano rappresentati usando numeri floating point (a 32 bit). In un modello quantizzato tutte le operazioni verranno eseguite con una precisione inferiore rispetto al caso floating point. Viene commesso quello che si chiama errore di quantizzazione. L'utilizzo di formati numerici a minore precisione consente di rappresentare una rete in maniera più compatta e di sfruttare operazioni "vettoriali" (o più generalmente fra tensori) ad elevate prestazioni supportate direttamente da molte piattaforme hardware. PyTorch supporta quantizzazione INT8 che consente, rispetto al caso standard FP32, una riduzione di 4x della dimensione del modello e della banda di memoria richiesta. Grazie al supporto diretto dell'hardware, le computazioni su modelli quantizzati INT8 risultano da 2 a 4 volte più rapide rispetto al caso FP32. Gli autori sostengono che la quantizzazione sia una tecnica per migliorare soprattutto la velocità di inferenza. Gli operatori quantizzati sono supportati solo nella fase forward-pass.

PyTorch supporta diversi approcci al processo di quantizzazione di un modello. Generalmente, un modello viene prima addestrato in FP32 e successivamente quantizzato, convertito, in INT8. PyTorch supporta anche la quantization aware training.

PyTorch supporta diverse rappresentazioni per i tensori quantizzati e supporta operazioni fra di essi. Fornisce una interfaccia (API) di alto livello che permette la quantizzazione di un modello con minimo degrado nell'accuratezza finale.

PyTorch fornisce tre diverse modalità di quantizzazione: 
+ Eager Mode Quantization
    + è una funzionalità beta. Gli utenti devono specificare manualmente i punti in cui eseguire la quantizzazione e la dequantizzazione. Sono    supportati solo i moduli (Module) e non anche i funzionali (functional)
+ FX Graph Model Quantization
    + è un processo automatico di quantizzazione in PyTorch. Si tratta di un prototipo ed attualmente è in modalità manutenzione poiché si dispone della modalità PyTorch 2 Export. Fornisce supporto anche ai funzionali ed automatizza il processo di quantizzazione. Tuttavia il modello potrebbe dover essere rifattorizzato affinché raggiunga la conformità alla modalità di quantizzazione FX Graph Model (il modello deve essere simbolicamente tracciabile) 
+ PyTorch 2 Export Quantization
    + ........

Sono supportati 3 tipi di quantizzazione:
1. Quantizzazione dinamica
2. Quantizzazione statica
3. Quantization aware training statica

Non tutti i tipi di quantizzazione supportano tutti i tipi di operatori. 

![](img\operator_support_quantization.png)

#### Quantizzazione dinamica post training
I pesi vengono quantizzati post training mentre le attivazioni vengono quantizzate dinamicamente durante l'inferenza. Conviene utilizzare questo schema di quantizzazione quando il tempo di inferenza dipende dal recupero dei pesi dalla memoria piuttosto che dal tempo necessario alla computazione della moltiplicazione fra matrici. L'input viene quantizzato al volo, si esegue la moltiplicazione fra il tensore dei pesi e l'input quantizzato, il risultato viene dequantizzato e passato alla funzione di attivazione la quale produrrà un risultato in FP32 che costituirà l'input verso lo strato successivo e quindi di nuovo riquantizzato al volo (ecco perché si parla di quantizzazione dinamica).

#### Quantizzazione statica post training 
In questo schema di quantizzazione (PTQ static) vengono quantizzati sia i pesi che le attivazioni. Richiede l'esecuzione di un processo di calibrazione, utilizzando un dataset rappresentativo, il cui fine è quello di stimare i parametri di quantizzazione (Scale, Zero-Point) da utilizzare per quantizzare le attivazioni. Solamente l'input esterno, quello proveniente dall'utente, deve necessariamente essere quantizzato dinamicamente, al volo, usando i parametri di quantizzazione stimati durante il processo di calibrazione post training. Tutto il resto, all'interno della rete, è rappresentato forma intera, sia i pesi che le attivazioni. Le prestazioni ci si aspetta che siano migliori rispetto al caso della quantizzazione dinamica poiché tutte le computazioni, nella rete, sono discrete mentre nel caso dinamico si rendono necessarie diverse operazioni di dequantizzazione e successiva riquantizzazione, il che comporta un carico computazionale ed un rallentamento nel processo di inferenza.

#### Quantization aware training per quantizzazione statica
Possiamo vederla come una speciale modalità di addestramento di una rete neurale che consente di considerare, già in fase di training, gli effetti della quantizzazione ed il cui risultato finale sarà un modello quantizzato staticamente. L'accuratezza del modello quantizzato è molto vicina a quella del modello originale non quantizzato.

#### Tensori quantizzati
Per poter quantizzare un modello in PyTorch, sono necessarie strutture per rappresentare dati quantizzati come tensori. I tensori quantizzati consentono di memorizzare i dati quantizzati (espressi come int8, uint8 oppure int32) insieme ai parametri di quantizzazione ovvero il fattore di scala S e lo zero-point Z. I tensori quantizzati consentono di semplificare l'esecuzione di operazioni fra di essi. PyTorch supporta schemi di quantizzazione per-Tensor e per-Channel sia simmetriche che asimmetrica (o affine). Per-Tensor significa che tutti i valori di un tensore vengono quantizzati utilizzando gli stessi paramentri di quantizzazione. Per-Channel significa invece che per ogni dimensione, tipicamente una canale (che per strati lineari corrispondono all'output dei neuroni), i valori di un tensore vengono quantizzati utilizzando parametri di quantizzazione diversi. La quantizzazione risultante sarà più precisa.

#### Tipologie di quantizzazione
La tabella sottostante riassume le diverse tipologie di quantizzazione supportate da PyTorch

![](img\quantization_mode_support.png)
<br><br>
#### Support Backend/Hardware 

![](img\hardware_backend_support.png)

PyTorch supporta i seguenti backends per l'esecuzione efficiente di tutte quelle operazioni fra tensori quantizzati
+ CPU x86 attraverso le librerie fbgemm e onednn
    + fbgemm (Facebook General Matrix-Matrix Multiplication) è una libreria che consente di eseguire, con elevate prestazioni, moltiplicazioni fra matrici e convoluzioni utili durante l'inferenza
    + oneDNN (OneAPI Deep Neural Network) è una libreria open source, ottimizzata per diverse architetture, che consente di migliorare le prestazioni sia su CPU che GPU#### Supporto degli operatori
+ CPU ARM, tipicamente installate nei dispositivi mobili o dispositivi embedded, attraverso qnnpack
    + qnnpack (Quantized Neural Networks Package) è una libreria ottimizzata per architetture mobili e fornisce implementazioni di operazioni su reti neurali ad elevate prestazioni su tensori quantizzati usando 8 bit. Il suo livello di astrazione è basso e dunque non creata per essere utilizzata direttamente ma per costruire, sopra di essa, ulteriori librerie/framework di livello più alto.
+ NVIDIA GPU attraverso TensorRT

#### Supporto degli operatori
Gli operatori supportati dipendono dalla tipologia di quantizzazione, statica o dinamica. Segue una tabella che riporta gli operatori supportati in funzione della tipologia di quantizzazione

![operator_support](img\operator_support.png)

